# Using Graph Theory to Model Chemical Reaction Networks

*By Dr. Tyler Biggs*

---

## Consider a Chemical Reaction Network

---



## In Python with NetworkX

---

To model a network of chemical reactions we need a multi-e

In [103]:
import networkx as nx
import numpy as np
import scipy
import tqdm
import matplotlib.pyplot as plt
import itertools
import collections
from scipy.special import factorial
%matplotlib inline

### Self-Equilibriating Reactions

Given by the equations:

$$ \bar{X} + Y \xrightarrow{c_1} 2Y $$

$$ 2Y \xrightarrow{c_2} Z $$

Where $\bar{X}$ is held constant, effectively representing an inexhaustible supply.

Where $S$ is the **stoichiometric matrix**.


$
S = 
\begin{bmatrix}
-1  &  1  &  0 \\
 0  & -2  &  1 \\
\end{bmatrix}
$

## Graph of Complexes

A chemical reaction network cannot be captured by an ordinary graph.

> **Complexes** Are the union of all the unique left and right hand sides of each reaction in the network.

There are $m$ chemical species (columns).
Each row is a column.

$
Z = 
\begin{bmatrix}
 1  &  1  &  0  \\
 0  &  2  &  0  \\
 0  &  0  &  1  \\
\end{bmatrix}
$

**Enumerate Complexes**

$$
X + Y \xrightarrow{c_1} 2Y \\
2Y \xrightarrow{c_2} Z
$$

In [104]:
lotka_graph = nx.DiGraph()

### Create the nodes

In [105]:
# Nodes can be any Python object, in this case I have chosen tuples.
complexes = {
    1: ('X', 'Y'),
    2: ('Y', 'Y'),
    3: ('Z')
}

# rxn_complex_2 = {
#     1: ('A')
# }
lotka_graph.add_nodes_from(complexes.keys())

### Complex-Stoichiometric Matrix

In [106]:
def build_complex_stoichiometric_matrix(complexes):
    """
    """
    # Create a list to store the output matrix.
    output = list()
    
    # Create a set, which will contain all the unique compounds.
    # chain.from_iterable unpacks, or flattens, nested lists.
    species = sorted(set(itertools.chain.from_iterable(complexes.values())))
    
    for com in complexes.values():
        
        # Create a counter to count the occurences of species
        # within each complex.
        count = collections.Counter(com)
        
        # Create the stoichiometry row for this complex, and append it
        # to the output list.
        row = [count[s] for s in species]
        output.append(row)
    
    # Return the transpose of a numpy matrix generated from the output list.
    return np.matrix(output).T

In [107]:
complex_stoich_matrix = Z = build_complex_stoichiometric_matrix(complexes)
complex_stoich_matrix

matrix([[1, 0, 0],
        [1, 2, 0],
        [0, 0, 1]])

In [108]:
complex_stoich_matrix[0, 0] = 0
complex_stoich_matrix

matrix([[0, 0, 0],
        [1, 2, 0],
        [0, 0, 1]])

### Add the edges

Stoichiometry given by the number of edges.

---

In [109]:
# Add the edges between each member on the left hand side of a reaction
# to the right hand side. Create the same number of edges as the
# reaction coefficient. We will label them by the reaction constant tag.
c1 = lotka_graph.add_edge(1, 2, rxn='c1', rate=5.0)
c2 = lotka_graph.add_edge(2, 3, rxn='c2', rate=0.005)
lotka_graph.edges()

OutEdgeView([(1, 2), (2, 3)])

In [110]:
incidence_matrix = B = scipy.sparse.csr_matrix.todense(
    nx.incidence_matrix(  # Assigns each row to a node and each column to an edge.
        lotka_graph, 
        oriented=True,  # Enable +1 or -1 based on edge direction.
        nodelist=sorted(complexes.keys())
    ))
incidence_matrix

matrix([[-1.,  0.],
        [ 1., -1.],
        [ 0.,  1.]])

In [111]:
stoich_matrix = S = complex_stoich_matrix * incidence_matrix
stoich_matrix

matrix([[ 0.,  0.],
        [ 1., -2.],
        [ 0.,  1.]])

In [112]:
np.dot(complex_stoich_matrix, incidence_matrix)

matrix([[ 0.,  0.],
        [ 1., -2.],
        [ 0.,  1.]])

### Define Rate Constants

These are arbitrary constants, use whatever coefficient of time desired.

In [113]:
species = x = np.matrix([5000, 3000, 1]).T
x

matrix([[5000],
        [3000],
        [   1]])

### Mass Action Kinetics

*Modeling the relation between the reaction rates and species concentrations.*

---

$$
v_j(x) = k_j e^{Z^{T}_{S_j} ln(x)}
$$

Where:

+ $v$ : a vector of reaction rates.
+ $x$ : denotes the species concentration vector.
+ $j$ : denotes the j-th reaction.
+ $Z$ : the complex-stoichiometric matrix.
+ $S$ : Stoichiometric matrix.
+ $Z_{S_j}$ : A column of the complex-stoichiometric matrix corresponding to the substrate complex j.
+ $T$ : Denotes the transpose of this matrix.

In [114]:
rates = nx.get_edge_attributes(lotka_graph, 'rate')
rates[(1,2)]

5.0

In [115]:
Z

matrix([[0, 0, 0],
        [1, 2, 0],
        [0, 0, 1]])

In [101]:
for idx, rate in enumerate(rates.values()):

    
    print(rate * np.exp(Z[:, idx].T * np.log(x)))

[[15000.]]
[[45000.]]


In [ ]:
def gillespie_simulaste():
    
    # Calculate the mass-action.
    
    # Calculate the sum of the mass-action.
    
    # Create two random numbers.
    
    # Calculate tau.
    
    # Calculate mu.
    
    

#### References

1. https://arxiv.org/pdf/1211.6643.pdf
2. http://www-users.math.umn.edu/~othmer/graphrt.pdf